In [4]:
import paramiko

ip = "172.16.16.122"   # 원격 서버 IP
user = "root"
pw = "asd123!@"
nfs_ip =  "172.16.16.122"  ## nfs를 사용할 ip

def cc(cmd):
    """SSH로 명령을 실행하고 결과를 출력"""
    cli = paramiko.SSHClient()
    cli.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    cli.connect(ip, username=user, password=pw, port=22)
    stdin, stdout, stderr = cli.exec_command(cmd)
    out = stdout.read().decode()
    err = stderr.read().decode()
    cli.close()
    if err:
        print("⚠️ 오류 발생:", err)
    return out.strip()

def nfs_install():
    print("=== NFS 설치 및 공유 디렉토리 생성 ===")
    cc("dnf -y install nfs-utils")
    cc("mkdir -p /mnt/nfs_share")
    # exports 설정
    cc(f'echo "/mnt/nfs_share {nfs_ip}(rw,sync,no_root_squash)" >> /etc/exports')
    print("✅ NFS 설치 완료")

def nfs_service():
    print("=== NFS 서비스 시작 ===")
    cc("systemctl enable nfs-server")
    cc("systemctl start nfs-server")
    cc("exportfs -v")  # 공유 확인
    print("✅ NFS 서비스 활성화 완료")

## 클라에서만 설치
def install_mount(server_ip):
    print("=== NFS 클라이언트 마운트 ===")
    cc("dnf -y install nfs-utils")   # 클라이언트에도 설치 필요
    cc("mkdir -p /mnt/nfs_data")
    cc(f"mount -t nfs {server_ip}:/mnt/nfs_share /mnt/nfs_data")
    print("✅ NFS 마운트 완료")

def main():
    nfs_install()
    nfs_service()
    install_mount(nfs_ip)

main()


=== NFS 설치 및 공유 디렉토리 생성 ===
✅ NFS 설치 완료
=== NFS 서비스 시작 ===
✅ NFS 서비스 활성화 완료
=== NFS 클라이언트 마운트 ===
⚠️ 오류 발생: mount.nfs: mounting 172.16.16.100:/mnt/nfs_share failed, reason given by server: No such file or directory

✅ NFS 마운트 완료
